Wine quality through hand made simple models.

In [37]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv


In [38]:
train = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [39]:
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Lets start with a linear model

In [40]:
import torch

creating the simplest linear model

In [41]:

def model(t_u, w, b):
    return w * t_u + b

initialising w and b

In [42]:

params = torch.tensor([1.0, 0.0], requires_grad=True)

lets get the tensors out of training data

### getting our data

In [43]:

t_u = train.drop('quality', axis=1)
t_c = train.quality

these are our x'es - input

In [44]:
t_u

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


`t_c` is our target

In [45]:
t_c 

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

### preprocessing

`describe` function helps us to know more about our dataset. look at the mean column, we want it to be in the similar range

In [46]:
t_u.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000


We see that that the columns are not in the same mean range so we need to normalise the data first.

using mean normalisation

In [47]:
t_un =(t_u-t_u.mean())/t_u.std()

In [48]:
t_un

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946
1,-0.298454,1.966827,-1.391037,0.043403,0.223805,0.872365,0.624168,0.028252,-0.719708,0.128910,-0.584594
2,-0.298454,1.296660,-1.185699,-0.169374,0.096323,-0.083643,0.228975,0.134222,-0.331073,-0.048074,-0.584594
3,1.654339,-1.384011,1.483689,-0.453077,-0.264878,0.107558,0.411372,0.664069,-0.978798,-0.461036,-0.584594
4,-0.528194,0.961576,-1.391037,-0.453077,-0.243630,-0.466047,-0.379014,0.558100,1.288240,-0.579025,-0.959946
...,...,...,...,...,...,...,...,...,...,...,...
1594,-1.217415,0.403103,-0.980362,-0.382151,0.053829,1.541571,-0.075020,-0.978459,0.899605,-0.461036,0.072271
1595,-1.389721,0.123866,-0.877693,-0.240300,-0.541090,2.210777,0.137777,-0.861893,1.353012,0.600867,0.729136
1596,-1.159980,-0.099523,-0.723690,-0.169374,-0.243630,1.254769,-0.196617,-0.533387,0.705287,0.541872,0.541460
1597,-1.389721,0.654416,-0.775024,-0.382151,-0.264878,1.541571,-0.075020,-0.676446,1.676875,0.305894,-0.209243


In [49]:
t_un.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03
mean,-1.570643e-14,-1.590973e-15,6.045057e-15,-1.344696e-15,4.924731e-15,-7.724347e-17,1.619856e-16,4.946064e-13,-5.937923e-15,-2.175036e-15,2.580411e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.136377e+00,-2.277567e+00,-1.391037e+00,-1.162333e+00,-1.603443e+00,-1.422055e+00,-1.230199e+00,-3.537625e+00,-3.699244e+00,-1.935902e+00,-1.898325e+00
25%,-7.004996e-01,-7.696903e-01,-9.290275e-01,-4.530767e-01,-3.711129e-01,-8.484502e-01,-7.438076e-01,-6.075656e-01,-6.549356e-01,-6.380200e-01,-8.661079e-01
50%,-2.410190e-01,-4.367545e-02,-5.634264e-02,-2.402999e-01,-1.798892e-01,-1.792441e-01,-2.574163e-01,1.759533e-03,-7.210449e-03,-2.250577e-01,-2.092427e-01
75%,5.056370e-01,6.264921e-01,7.650078e-01,4.340257e-02,5.382858e-02,4.899619e-01,4.721707e-01,5.766445e-01,5.757422e-01,4.238832e-01,6.352984e-01
max,4.353787e+00,5.876138e+00,3.742403e+00,9.192806e+00,1.112355e+01,5.365606e+00,7.372847e+00,3.678904e+00,4.526866e+00,7.916200e+00,4.201138e+00


using min max normalisation****

In [50]:
t_un2 = (t_u-t_u.min())/(t_u.max()-t_u.min())

In [51]:
t_un2.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,0.329171,0.279329,0.270976,0.112247,0.125988,0.209506,0.142996,0.490211,0.449695,0.196496,0.311228
std,0.154079,0.122644,0.194801,0.096570,0.078573,0.147326,0.116238,0.138571,0.121564,0.101501,0.163949
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.221239,0.184932,0.090000,0.068493,0.096828,0.084507,0.056537,0.406021,0.370079,0.131737,0.169231
50%,0.292035,0.273973,0.260000,0.089041,0.111853,0.183099,0.113074,0.490455,0.448819,0.173653,0.276923
75%,0.407080,0.356164,0.420000,0.116438,0.130217,0.281690,0.197880,0.570117,0.519685,0.239521,0.415385
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
t_un2

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846
...,...,...,...,...,...,...,...,...,...,...,...
1594,0.141593,0.328767,0.08,0.075342,0.130217,0.436620,0.134276,0.354626,0.559055,0.149701,0.323077
1595,0.115044,0.294521,0.10,0.089041,0.083472,0.535211,0.159011,0.370778,0.614173,0.257485,0.430769
1596,0.150442,0.267123,0.13,0.095890,0.106845,0.394366,0.120141,0.416300,0.535433,0.251497,0.400000
1597,0.115044,0.359589,0.12,0.075342,0.105175,0.436620,0.134276,0.396476,0.653543,0.227545,0.276923


I personally like min max normalisation.
Note that we are not normalising `t_c`. (should we?)
 

### creating a validation set

we need to do this before we get into the training loop

In [53]:
t_un2 = torch.tensor(t_un2.values)
t_c = torch.tensor(t_c.values)

In [54]:
n_samples = t_un2.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices[:5], val_indices[:5]

(tensor([1490, 1519,  568, 1313, 1480]),
 tensor([1336,  302,  187, 1094,   76]))

In [55]:
t_u_train = t_un2[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_un2[val_indices]
t_c_val = t_c[val_indices]

### creating tensors

Next step is to convert `t_un2` and `t_c` into tensors of right format to be used as input to the linear model.


In [56]:

t_u_train = t_u_train.unsqueeze(1)
t_c_train = t_c_train.unsqueeze(1)

t_u_val = t_u_val.unsqueeze(1)
t_c_val = t_c_val.unsqueeze(1)

lets ensure all of the tensors are float now.

In [59]:
t_u_train, t_u_val = t_u_train.type(torch.FloatTensor), t_u_val.type(torch.FloatTensor)
t_c_train, t_c_val = t_c_train.type(torch.FloatTensor), t_c_val.type(torch.FloatTensor)

In [57]:
t_u_train.shape, t_u_val.shape

(torch.Size([1280, 1, 11]), torch.Size([319, 1, 11]))

In [58]:
t_c_train.shape, t_c_val.shape

(torch.Size([1280, 1]), torch.Size([319, 1]))

one last thing would be that the input dimension is different from the target dimension. usually pytorch will give a broadcasting warning for this, but lets decide to use the tensor as this.

So our preprocessing of tensors is complete.

### Now comes the training loop.

In [103]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = torch.optim.SGD([params], lr=learning_rate)

In [102]:
def training_loop(n_epochs=3000, optimizer= optimizer, params=params,train_t_u= t_u_train, val_t_u=t_u_val, train_t_c=t_c_train, val_t_c=t_c_val, loss_fn=torch.nn.MSELoss(),model=model):
    for epoch in range(1, n_epochs+1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print("Epoch: ", epoch, " Training loss: ", train_loss.item(), " Val loss ", val_loss.item())
    return params
    

In [104]:
optimal_params = training_loop()
%time

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1280, 1])) that is different to the input size (torch.Size([1280, 1, 11])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([319, 1])) that is different to the input size (torch.Size([319, 1, 11])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  1  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  2  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  3  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  500  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  1000  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  1500  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  2000  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  2500  Training loss:  0.6491010189056396  Val loss  0.664764940738678
Epoch:  3000  Training loss:  0.6491010189056396  Val loss  0.664764940738678
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 8.58 µs


It does take a lot of time. But lets try using a neural network next.

In [77]:
seq_model = torch.nn.Sequential(
    torch.nn.Linear(11,13),
    torch.nn.Tanh(),
    torch.nn.Linear(13,1),
)

seq_model

Sequential(
  (0): Linear(in_features=11, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

lets look at the parameters

In [78]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 11]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

Lets check all the sizes of parameters first.

In [79]:
t_u_train.shape, t_c_train.shape, t_u_val.shape, t_c_val.shape

(torch.Size([1280, 1, 11]),
 torch.Size([1280, 1]),
 torch.Size([319, 1, 11]),
 torch.Size([319, 1]))

In [82]:
def training_loop(n_epochs=3000, optimizer= optimizer, params=params,train_t_u= t_u_train, val_t_u=t_u_val, train_t_c=t_c_train, val_t_c=t_c_val, loss_fn=torch.nn.MSELoss(),model=model):
    for epoch in range(1, n_epochs+1):
#         train_t_p = model(train_t_u, *params)
        train_t_p = model(train_t_u)
        train_loss = loss_fn(train_t_p, train_t_c)
        
#         val_t_p = model(val_t_u, *params)
        val_t_p = model(val_t_u)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print("Epoch: ", epoch, " Training loss: ", train_loss.item(), " Val loss ", val_loss.item())
#     return params
    

In [83]:
optimizer = torch.optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model = seq_model,
    loss_fn=torch.nn.MSELoss(),
)

%time

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([319, 1])) that is different to the input size (torch.Size([319, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  1  Training loss:  35.84245681762695  Val loss  35.22545623779297
Epoch:  2  Training loss:  35.4724235534668  Val loss  34.85795974731445
Epoch:  3  Training loss:  35.108184814453125  Val loss  34.49624252319336
Epoch:  500  Training loss:  0.6962293982505798  Val loss  0.7094822525978088
Epoch:  1000  Training loss:  0.6914792656898499  Val loss  0.706536054611206
Epoch:  1500  Training loss:  0.6876910924911499  Val loss  0.7027801275253296
Epoch:  2000  Training loss:  0.6844990253448486  Val loss  0.699619710445404
Epoch:  2500  Training loss:  0.6817776560783386  Val loss  0.6969297528266907
Epoch:  3000  Training loss:  0.679433286190033  Val loss  0.6946167349815369
Epoch:  3500  Training loss:  0.6773956418037415  Val loss  0.6926096081733704
Epoch:  4000  Training loss:  0.675609827041626  Val loss  0.6908533573150635
Epoch:  4500  Training loss:  0.6740332841873169  Val loss  0.68930584192276
Epoch:  5000  Training loss:  0.6726321578025818  Val loss  0.687932908535

In [85]:
print('output', seq_model(t_u_val))

output tensor([[[5.2756]],

        [[5.7719]],

        [[5.5598]],

        [[5.5849]],

        [[5.8905]],

        [[5.6555]],

        [[5.5382]],

        [[5.8469]],

        [[5.4732]],

        [[5.4872]],

        [[5.9542]],

        [[5.4777]],

        [[5.5159]],

        [[5.7302]],

        [[5.8156]],

        [[5.6088]],

        [[5.8645]],

        [[5.5246]],

        [[5.6415]],

        [[5.7259]],

        [[5.9326]],

        [[5.8340]],

        [[5.6669]],

        [[5.5915]],

        [[5.7149]],

        [[5.7641]],

        [[5.6730]],

        [[5.7486]],

        [[5.5812]],

        [[5.8863]],

        [[5.4671]],

        [[5.7796]],

        [[5.4947]],

        [[5.7915]],

        [[5.4070]],

        [[5.7163]],

        [[5.3504]],

        [[5.6510]],

        [[5.6193]],

        [[5.8281]],

        [[6.0243]],

        [[5.6151]],

        [[5.7060]],

        [[5.9919]],

        [[5.5459]],

        [[5.8060]],

        [[5.7934]],

      

In [86]:
print('answer', t_c_val)

answer tensor([[5.],
        [5.],
        [5.],
        [6.],
        [5.],
        [7.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [5.],
        [6.],
        [5.],
        [5.],
        [7.],
        [5.],
        [5.],
        [6.],
        [5.],
        [5.],
        [5.],
        [6.],
        [6.],
        [7.],
        [5.],
        [5.],
        [5.],
        [5.],
        [5.],
        [5.],
        [7.],
        [6.],
        [4.],
        [6.],
        [7.],
        [6.],
        [7.],
        [5.],
        [5.],
        [6.],
        [5.],
        [5.],
        [4.],
        [6.],
        [5.],
        [5.],
        [5.],
        [6.],
        [5.],
        [5.],
        [6.],
        [7.],
        [7.],
        [5.],
        [6.],
        [6.],
        [5.],
        [5.],
        [5.],
        [5.],
        [6.],
        [5.],
        [6.],
        [5.],
        [5.],
        [5.],
        [7.],
        [5.],
        [7.],
        [7.],

Lets compare the two models

In [96]:
from matplotlib import pyplot as plt

t_range = torch.arange(20., 90.).unsqueeze(1)
fig = plt.figure(dpi=600)

try: 
#     plt.xlabel("X")
#     plt.ylabel("y")
#     plt.plot(t_u_train.numpy(), t_c_train.numpy(), 'o')
    plt.plot(t_range.numpy(), seq_model(0.1 * t_range).detach().numpy(), 'c-')
    plt.plot(t_u_train.numpy(), seq_model(0.1 * t_u_train).detach().numpy(), 'kx')
except Exception as e:
    print(e)

mat1 and mat2 shapes cannot be multiplied (70x1 and 11x13)


<Figure size 3600x2400 with 0 Axes>

It will be interesting if we finda way to plot all 11 values of x. Lets make some predictions.

In [98]:
len(t_u_val)

319

In [99]:
t_u_val[289]

tensor([[0.2124, 0.3151, 0.1200, 0.0685, 0.1319, 0.4648, 0.4170, 0.4060, 0.5512,
         0.0898, 0.3231]])

In [100]:
t_c_val[289]

tensor([5.])

Lets see what linear model gives

In [105]:
optimal_params

tensor([0.1060, 5.6179], requires_grad=True)

In [108]:
model(t_c_val[289], *optimal_params)

tensor([6.1478], grad_fn=<AddBackward0>)

Now the neural net model

In [109]:
seq_model(t_u_val[289])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1 and 11x13)